# YouTube & Spotify Top Music Artists From 2018
---

### SCOPE:
#### - Extracted, transformed, and loaded up YouTube's Top Trending Videos from December 2017 thru May 2018 for their videos categorized as music only, and created an "Artist" column to enable joining with Spotify's Top 100 Songs of 2018. Both dataframes were loaded into MySQL.


### PURPOSE:
#### - I choose this project because I'm a avid listener and a huge music and concert goer, and wanted to work with data that I was familiar with.

### Data Sources:
#### - https://www.kaggle.com/datasnaek/youtube-new (this is an updated link, whereas I used an older version of this file, which is attached in the resources)
#### - https://www.kaggle.com/nadintamer/top-spotify-tracks-of-2018
 


In [1]:
# Import Dependencies 1/2:
import os
import csv
import json
import simplejson
import numpy as np
import pandas as pd
from datetime import datetime
import sys
import string

## SQL Alchemy setup and built a search, create, and drop database function to set up loading phase after extraction and transformation of data

In [2]:
# Import Dependencies 2/2:
from sqlalchemy import create_engine, Column, Integer, String, join, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import database_exists, create_database, drop_database, has_index
import pymysql

Base = declarative_base()

In [3]:
#rds_connection_string = "<inser user name>:<insert password>@127.0.0.1/customer_db"
rds_connection_string = "root:gREATNESS23$@127.0.0.1/" #youtube_spotify_2018_db"

# Can set up an input for the db_name later (optional)
#db_name = input("What database would you like to search for?")
db_name = 'youtube_spotify_2018_db2'

# Setup engine connection string
engine = create_engine(f'mysql://{rds_connection_string}{db_name}?charset=utf8')

In [4]:
# Created a function incorproating SQL Alchemy to search, create, and or drop a database:
def search_create_drop_db(db_name):
    #db_exist = database_exists(f'mysql://{rds_connection_string}youtube_spotify_2018_db2')
    db_exist = database_exists(f'mysql://{rds_connection_string}{db_name}')
    db_url = f'mysql://{rds_connection_string}{db_name}'
    if db_exist == True:
        drop_table_y_or_n = input(f'"{db_name}" database already exists in MySQL. Do you want you drop the table? Enter exactly: "y" or "n".  ')
        if drop_table_y_or_n == 'y':
            drop_database(db_url)
            print(f"Database {db_name} was dropped")
            create_new_db = input(f"Do you want to create another database called: {db_name}?  ")
            if create_new_db == 'y':
                create_database(db_url)
                return(f"The database {db_name} was created. Next You will need to create tables for this database.  ")
            else:
                return("No database was created. Goodbye!  ")
        else:
            return("The database exists. No action was taken. Goodbye!  ")
    else:
        create_database(db_url)
        return(f"The queried database did not exist, and was created as: {db_name} .  ")

search_create_drop_db(db_name)

"youtube_spotify_2018_db2" database already exists in MySQL. Do you want you drop the table? Enter exactly: "y" or "n".  y
Database youtube_spotify_2018_db2 was dropped
Do you want to create another database called: youtube_spotify_2018_db2?  y


'The database youtube_spotify_2018_db2 was created. Next You will need to create tables for this database.  '

In [5]:
# Create tables 'blueprints' using python classes and sql alchemy:
# This would be useful say if I was to use SQL Alchemy to import a sqllite file into python

class yt_categories(Base):
    __tablename__ = 'yt_category_titles'
    id = Column(Integer, primary_key=True)
    category_title = Column(String(60))

class yt_statistics_data(Base):
    __tablename__ = 'yt_statistics'
    id = Column(Integer, primary_key=True)
    category_title = Column(String(60))
    trending_date = Column(Date, nullable=False)
    video_title = Column(String(200))
    channel_title = Column(String(100))
    category_id = Column(Integer)
    views = Column(Integer)
    likes = Column(Integer)
    dislikes = Column(Integer)
    view_count = Column(Integer)

class spotify_music_data(Base):
    __tablename__ = 'spotify_music'
    id = Column(Integer, primary_key=True)
    artist = Column(String(100))
    song_name = Column(String(200))
    spotify_unique_id = Column(String(100))


## Extract and transform "YouTube Top Trending Videos" dataset for USA

In [6]:
# YouTube data has two parts: 1) Categories information in JSON format
                            # 2) Top Trending US YouTube Videos in a CSV file

# Part 1) YouTube Categories are seperated in a json file
yt_json_file = './resources/youtube_US_category_id.json'
yt_rawjson_df = pd.read_json(yt_json_file)

In [7]:
# Extract the category id and category titles, and set them into a list

# for i in yt_rawjson_df['items']:
#     #print(i['id'])
#     print(i['id'] + ' | ' + i['snippet']['title'])
    

category_id = [int(i['id']) for i in yt_rawjson_df['items']]
category_title = [str(i['snippet']['title']) for i in yt_rawjson_df['items']]

# Create a dataframe of the category id and title for later use
category_id_title_df = pd.DataFrame({'category_id': category_id, 'category_title': category_title})
category_id_title_df.head()
category_id_title_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
category_id       32 non-null int64
category_title    32 non-null object
dtypes: int64(1), object(1)
memory usage: 592.0+ bytes


In [ ]:
# Drop Dulplicates and Sort by category_title
category_id_title_df.drop_duplicates(['category_id', 'category_title']).sort_values(by=['category_title'], ascending=False).head()

In [9]:
# Load category_id_title_df to MySQL with Pandas
category_id_title_df.to_sql('yt_categories', con=engine)

In [10]:
# Two ways in this notebook to pull the data directly from MySQL database
# Method 1) Use SQL Alchemy Engine - Result: successfully reads from MySQL Database:
engine.execute("SELECT * FROM yt_categories").fetchall()

[(0, 1, 'Film & Animation'),
 (1, 2, 'Autos & Vehicles'),
 (2, 10, 'Music'),
 (3, 15, 'Pets & Animals'),
 (4, 17, 'Sports'),
 (5, 18, 'Short Movies'),
 (6, 19, 'Travel & Events'),
 (7, 20, 'Gaming'),
 (8, 21, 'Videoblogging'),
 (9, 22, 'People & Blogs'),
 (10, 23, 'Comedy'),
 (11, 24, 'Entertainment'),
 (12, 25, 'News & Politics'),
 (13, 26, 'Howto & Style'),
 (14, 27, 'Education'),
 (15, 28, 'Science & Technology'),
 (16, 29, 'Nonprofits & Activism'),
 (17, 30, 'Movies'),
 (18, 31, 'Anime/Animation'),
 (19, 32, 'Action/Adventure'),
 (20, 33, 'Classics'),
 (21, 34, 'Comedy'),
 (22, 35, 'Documentary'),
 (23, 36, 'Drama'),
 (24, 37, 'Family'),
 (25, 38, 'Foreign'),
 (26, 39, 'Horror'),
 (27, 40, 'Sci-Fi/Fantasy'),
 (28, 41, 'Thriller'),
 (29, 42, 'Shorts'),
 (30, 43, 'Shows'),
 (31, 44, 'Trailers')]

In [11]:
# Method 2) Read from MySQL database using Pandas - Result: Success! - This method is a better setup for analysis.
# The index column is automatically generated.
pd.read_sql_query('SELECT * FROM yt_categories', con=engine).head()

,index,category_id,category_title
0,0,1,Film & Animation
1,1,2,Autos & Vehicles
2,2,10,Music
3,3,15,Pets & Animals
4,4,17,Sports


In [12]:
# Part 1) is the YouTube Top US Videos in a CSV
csv_file_yt = "./resources/youtube_USvideos.csv"
yt_rawdata_df = pd.read_csv(csv_file_yt, encoding='utf-8')

In [13]:
# view rows, count and datatypes
yt_rawdata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
video_id                  40949 non-null object
trending_date             40949 non-null object
title                     40949 non-null object
channel_title             40949 non-null object
category_id               40949 non-null int64
publish_time              40949 non-null object
tags                      40949 non-null object
views                     40949 non-null int64
likes                     40949 non-null int64
dislikes                  40949 non-null int64
comment_count             40949 non-null int64
thumbnail_link            40949 non-null object
comments_disabled         40949 non-null bool
ratings_disabled          40949 non-null bool
video_error_or_removed    40949 non-null bool
description               40379 non-null object
dtypes: bool(3), int64(5), object(8)
memory usage: 4.2+ MB


In [14]:
# Rename Columns
yt_statistics_data = yt_rawdata_df.rename(columns={"video_id":"Video ID", "trending_date":"Trending Date",
                                                "title":"Title", "channel_title":"Channel Title",
                                                "category_id":"category_id", "publish_time":"Publish Time",
                                                "tags":"Tags", "views":"Views",
                                                "likes":"Likes", "dislikes":"Dislikes", 
                                                "comment_count":"Comment Count", "thumbnail_link":"Thumbnail Link",
                                                "comments_disabled":"Comments Disabled", "ratings_disabled":"Ratings Disabled",
                                                "video_error_or_removed":"Video Error Or Removed", "description":"Description"
                                               })
yt_statistics_data.head()

,Video ID,Trending Date,Title,Channel Title,category_id,Publish Time,Tags,Views,Likes,Dislikes,Comment Count,Thumbnail Link,Comments Disabled,Ratings Disabled,Video Error Or Removed,Description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [15]:
# Drop Cells with Missing Information
yt_statistics_data = yt_statistics_data.dropna(how="any")

In [16]:
# Drop Dulplicates and Sort by Trending Date
yt_statistics_data.drop_duplicates(['Video ID', 'Trending Date', 'Title', 'Channel Title', 'category_id', 'Publish Time']).sort_values(by=['Trending Date'], ascending=False).head()

,Video ID,Trending Date,Title,Channel Title,category_id,Publish Time,Tags,Views,Likes,Dislikes,Comment Count,Thumbnail Link,Comments Disabled,Ratings Disabled,Video Error Or Removed,Description
38051,EqeIRzY7hIU,18.31.05,6 Cheese Gadgets put to the Test!,CrazyRussianHacker,28,2018-05-20T18:58:15.000Z,"Cheese Gadgets|""Gadgets""|""Cheese""|""kitchen gad...",1519038,26770,1557,2875,https://i.ytimg.com/vi/EqeIRzY7hIU/default.jpg,False,False,False,$1000 Survival Kit in a Case - https://youtu.b...
38103,MAjY8mCTXWk,18.31.05,"周杰倫 Jay Chou【不愛我就拉倒 If You Don't Love Me, It's...",杰威爾音樂 JVR Music,10,2018-05-14T15:59:47.000Z,"周杰倫|""Jay""|""Chou""|""周董""|""周杰伦""|""周傑倫""|""杰威尔""|""周周""|""...",17259071,132009,9552,14789,https://i.ytimg.com/vi/MAjY8mCTXWk/default.jpg,False,False,False,詞：周杰倫、宋健彰（彈頭） 曲：周杰倫MV導演：周杰倫憂鬱型男的走心旋律 用英式搖滾宣洩...
38081,LzuDyq0-1LM,18.31.05,Why it's not a British royal wedding without f...,Vox,25,2018-05-18T11:00:03.000Z,"pop culture|""royal wedding""|""fancy hats""|""prin...",484199,8246,662,761,https://i.ytimg.com/vi/LzuDyq0-1LM/default.jpg,False,False,False,Fantastical fascinators at royal weddings are ...
38082,6SuMbFuKDf8,18.31.05,Backstreet Boys - Don't Go Breaking My Heart (...,BackstreetBoysVEVO,10,2018-05-17T04:00:01.000Z,"Backstreet Boys|""Don't Go Breaking My Heart""|""...",14717193,396826,16015,39035,https://i.ytimg.com/vi/6SuMbFuKDf8/default.jpg,False,False,False,Get the Backstreet Boys new single “Don’t Go B...
38083,yDiXQl7grPQ,18.31.05,Do You Hear Yanny or Laurel? (SOLVED with SCIE...,AsapSCIENCE,28,2018-05-16T18:16:26.000Z,"AsapSCIENCE|""audio illusion""|""yanny""|""laurel""|...",42667467,564046,33508,180469,https://i.ytimg.com/vi/yDiXQl7grPQ/default.jpg,False,False,False,Yanny vs. Laurel audio illusion solved! PHEW F...


In [17]:
# Drop Unwanted Columns

to_drop =['Publish Time', 'Tags', 'Thumbnail Link', 'Comments Disabled', 'Ratings Disabled', 'Video Error Or Removed', 'Description']

yt_statistics_data.drop(to_drop, inplace=True, axis=1)

In [18]:
# Replace the "." in Trending Date to "-"

yt_statistics_data['Trending Date'] = [x.replace(".","-") for x in yt_statistics_data['Trending Date']]
yt_statistics_data.head()

,Video ID,Trending Date,Title,Channel Title,category_id,Views,Likes,Dislikes,Comment Count
0,2kyS6SvSYSE,17-14-11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,748374,57527,2966,15954
1,1ZAPwfrtAFY,17-14-11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2418783,97185,6146,12703
2,5qpjK5DgCt4,17-14-11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,3191434,146033,5339,8181
3,puqaWrEC7tY,17-14-11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,343168,10172,666,2146
4,d380meD0W0M,17-14-11,I Dare You: GOING BALD!?,nigahiga,24,2095731,132235,1989,17518


In [19]:
# Pulled up this dataframe verify exactly how "Music" was spelled for the following step:
category_id_title_df['category_title'].value_counts()

Comedy                   2
Sci-Fi/Fantasy           1
Classics                 1
Documentary              1
People & Blogs           1
Movies                   1
Anime/Animation          1
Pets & Animals           1
Shorts                   1
Gaming                   1
Autos & Vehicles         1
Thriller                 1
Trailers                 1
Shows                    1
Entertainment            1
Family                   1
Action/Adventure         1
Education                1
Film & Animation         1
Science & Technology     1
Short Movies             1
Foreign                  1
Horror                   1
Videoblogging            1
Howto & Style            1
Drama                    1
Nonprofits & Activism    1
News & Politics          1
Sports                   1
Music                    1
Travel & Events          1
Name: category_title, dtype: int64

In [20]:
# Used Pandas Merge aka (Join in Relational DB) Inner joined the Both YouTube Tables to find what Category_id 10 means:

yt_merged_df = pd.merge(yt_statistics_data, category_id_title_df, how='inner', on='category_id',
         left_index=False, right_index=False, sort=False)

yt_musicdata_df = yt_merged_df[yt_merged_df['category_title']  == 'Music']

In [21]:
# Only going to clean and focus on YT videos that are in the "Music" category_title:

# Clean Channel Title Column to set up the MAIN LOOP

yt_musicdata_df['Channel Title'] = [x.replace("VEVO","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("vevo","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("Vevo","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("Official","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("official","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("OFFICIAL","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("You Tube Channel","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("Music","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace("music","") for x in yt_musicdata_df['Channel Title']]
yt_musicdata_df['Channel Title'] = [x.replace(" - Topic","") for x in yt_musicdata_df['Channel Title']]

yt_musicdata_df['Channel Title'].replace("BackstreetBoys","Backstreet Boys")
yt_musicdata_df['Channel Title'].replace("CalumScott","Calum Scott")
yt_musicdata_df['Channel Title'].replace("TaylorSwift","Taylor Swift")
yt_musicdata_df['Channel Title'].replace("NickiMinajAt","Nicki Minaj")
yt_musicdata_df['Channel Title'].replace("FifthHarmony","FifthHarmony")
yt_musicdata_df['Channel Title'].replace("davematthewsband","Dave Matthews Band")
yt_musicdata_df['Channel Title'].replace("EnriqueIglesias","Enrique Iglesias")
yt_musicdata_df['Channel Title'].replace("ChildishGambino","Childish Gambino")
yt_musicdata_df['Channel Title'].replace("SamSmithWorld","Sam Smith")
yt_musicdata_df['Channel Title'].replace("MeghanTrainor","Meghan Trainor")
yt_musicdata_df['Channel Title'].replace("johnmayer","John Mayer")
yt_musicdata_df['Channel Title'].replace("weezer","Weezer")
yt_musicdata_df['Channel Title'].replace("AzealiaBanks","Azealia Banks")
yt_musicdata_df['Channel Title'].replace("Maroon5","Maroon 5")
yt_musicdata_df['Channel Title'].replace("Zayn","ZAYN")
yt_musicdata_df['Channel Title'].replace("ArianaGrande","Ariana Grande")
yt_musicdata_df['Channel Title'].replace("CAguilera","Christina Aguilera")
yt_musicdata_df['Channel Title'].replace("LadyGaga","Lady Gaga")
yt_musicdata_df['Channel Title'].replace("ToniBraxton","Toni Braxton")
yt_musicdata_df['Channel Title'].replace("JasonAldean","Jason Aldean")
yt_musicdata_df['Channel Title'].replace("PTXofficial","PTX")
yt_musicdata_df['Channel Title'].replace("KeithUrban","Keith Urban")
yt_musicdata_df['Channel Title'].replace("KaceyMusgraves","Kacey Musgraves")
yt_musicdata_df['Channel Title'].replace("ChrisStapleton","Chris Stapleton")
yt_musicdata_df['Channel Title'].replace("ThirtySecondsToMars","Thirty Seconds To Mars")

# Clean Special Characters to prevent latin-1 encoding errors. Went back up to the pd.read_csv
# and added "encoding="utf-8"

yt_musicdata_df['Title'] = [x.replace("é","e") for x in yt_musicdata_df['Title']]
yt_musicdata_df['Title'] = [x.replace("ú","u") for x in yt_musicdata_df['Title']]
yt_musicdata_df['Title'] = [x.replace("®","") for x in yt_musicdata_df['Title']]


C:\Users\JOHNE\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\JOHNE\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\JOHNE\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [22]:
yt_musicdata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6437 entries, 25550 to 31986
Data columns (total 10 columns):
Video ID          6437 non-null object
Trending Date     6437 non-null object
Title             6437 non-null object
Channel Title     6437 non-null object
category_id       6437 non-null int64
Views             6437 non-null int64
Likes             6437 non-null int64
Dislikes          6437 non-null int64
Comment Count     6437 non-null int64
category_title    6437 non-null object
dtypes: int64(5), object(5)
memory usage: 553.2+ KB


In [23]:
# Load category_id_title_df to MySQL with Pandas
yt_musicdata_df.to_sql('yt_statistics_data', con=engine)

In [24]:
# Two ways in this notebook to pull the data directly from MySQL database
# Method 1) Use SQL Alchemy Engine - Result: successfully reads from MySQL Database:
engine.execute("SELECT * FROM yt_statistics_data").fetchall()

[(25550, '5E4ZBSInqUU', '17-14-11', 'Marshmello - Blocks (Official Music Video)', 'marshmello', 10, 687582, 114188, 1333, 8371, 'Music'),
 (25551, 'n1WpP7iowLc', '17-14-11', 'Eminem - Walk On Water (Audio) ft. Beyonce', 'Eminem', 10, 17158531, 787419, 43420, 125882, 'Music'),
 (25552, 'e_7zHm7GsYc', '17-14-11', 'Hunter Hayes - You Should Be Loved (Part One Of Pictures)', 'Hunter Hayes', 10, 13917, 1318, 24, 76, 'Music'),
 (25553, 'zZ9FciUx6gs', '17-14-11', 'Nickelback - The Betrayal Act III [Official Video]', 'Nickelback', 10, 57169, 6927, 161, 565, 'Music'),
 (25554, 'PaJCFHXcWmM', '17-14-11', 'U2 - The Blackout', 'U2', 10, 60506, 5389, 106, 455, 'Music'),
 (25555, '0tO_l_Ed5Rs', '17-14-11', 'Matthew Santoro - FACTS (Official Music Video) f. Ellevan & Humble the Poet', 'MatthewSantoro', 10, 328330, 15186, 15448, 7484, 'Music'),
 (25556, '9t9u_yPEidY', '17-14-11', 'Jennifer Lopez - Amor, Amor, Amor (Official Video) ft. Wisin', 'JenniferLopez', 10, 9548677, 190083, 15015, 11473, 'Music'

In [25]:
# Method 2) Read from MySQL database using Pandas - Result: Success! - This method is a better setup for analysis.
# The index column is automatically generated.
pd.read_sql_query('SELECT * FROM yt_statistics_data', con=engine).head()

,index,Video ID,Trending Date,Title,Channel Title,category_id,Views,Likes,Dislikes,Comment Count,category_title
0,25550,5E4ZBSInqUU,17-14-11,Marshmello - Blocks (Official Music Video),marshmello,10,687582,114188,1333,8371,Music
1,25551,n1WpP7iowLc,17-14-11,Eminem - Walk On Water (Audio) ft. Beyonce,Eminem,10,17158531,787419,43420,125882,Music
2,25552,e_7zHm7GsYc,17-14-11,Hunter Hayes - You Should Be Loved (Part One O...,Hunter Hayes,10,13917,1318,24,76,Music
3,25553,zZ9FciUx6gs,17-14-11,Nickelback - The Betrayal Act III [Official Vi...,Nickelback,10,57169,6927,161,565,Music
4,25554,PaJCFHXcWmM,17-14-11,U2 - The Blackout,U2,10,60506,5389,106,455,Music


## Extract and transform "Spotify Top 2018 Songs" dataset

In [ ]:
# Spotify 2018 - Top 100 Songs - Raw CSV

csv_file_spotify2018 = "./resources/spotify_top2018.csv"
spotify2018_rawdata_df = pd.read_csv(csv_file_spotify2018)
spotify2018_rawdata_df.head()

In [ ]:
spotify2018_rawdata_df.info()

In [ ]:
# Set up Spotify DataFrame
spotify_2018_id = spotify2018_rawdata_df['id']
spotify_2018_name = spotify2018_rawdata_df['name']
spotify_2018_artists = spotify2018_rawdata_df['artists']

In [ ]:
spotify2018_filtered_df = pd.DataFrame({
            "Artist": spotify_2018_artists,
            "Song Name": spotify_2018_name,
            "Spotify Unique ID": spotify_2018_id
             })
spotify2018_filtered_df.head()